# Creating data for pareto

## Load libraries and merge the df

In [7]:
import pandas as pd
import os
import shutil
path = os.chdir('C:/Users/jewel.espiritu/OneDrive - AMDATEX LAS PINAS SERVICES INC/Python/py') 
sladf = pd.concat(pd.read_excel('./data/WeeklySLA_Data_2022.xlsx', sheet_name=None), ignore_index=True)

## Create Pivot table

Filter the data that is older than 14 days

In [8]:
SLApivot = pd.pivot_table(sladf, index=['Service', 'Olderthan14', 'Week','TTR Result\n(Excluding Pending Time)'], columns=['Operational Status'], values=['Ticket No'], aggfunc='count')
paretopvt = SLApivot.query('Olderthan14=="Y"')
paretopvt

Ticket No  \
Operational Status                                                                                        On-going   
Service                                            Olderthan14 Week TTR Result\n(Excluding Pending Time)             
3rd Party/ Client Controlled Software, Network ... Y           1    Beyond TAT                                50.0   
                                                                    Within TAT                                10.0   
                                                               2    Beyond TAT                                45.0   
                                                                    Within TAT                                10.0   
                                                               3    Beyond TAT                                44.0   
...                                                                                                            ...   
WFH PC setup                                       Y           23   Within TAT                                 8.0   
                                                               24   Beyond TAT                                 5.0   
                                                                    Within TAT                                 3.0   
                                                               25   Beyond TAT                                 6.0   
                                                                    Within TAT                                 4.0   

                                                                                                                  \
Operational Status                                                                                       Pending   
Service                                            Olderthan14 Week TTR Result\n(Excluding Pending Time)           
3rd Party/ Client Controlled Software, Network ... Y           1    Beyond TAT                              11.0   
                                                                    Within TAT                              62.0   
                                                               2    Beyond TAT                               9.0   
                                                                    Within TAT                              36.0   
                                                               3    Beyond TAT                              12.0   
...                                                                                                          ...   
WFH PC setup                                       Y           23   Within TAT                              10.0   
                                                               24   Beyond TAT                               1.0   
                                                                    Within TAT                               4.0   
                                                               25   Beyond TAT                               NaN   
                                                                    Within TAT                               3.0   

                                                                                                                           
Operational Status                                                                                       Resolved/ Closed  
Service                                            Olderthan14 Week TTR Result\n(Excluding Pending Time)                   
3rd Party/ Client Controlled Software, Network ... Y           1    Beyond TAT                                        NaN  
                                                                    Within TAT                                        NaN  
                                                               2    Beyond TAT                                        NaN  
                                                                    Within TAT     

## Drop other columns

In [9]:
paretopvt.columns = paretopvt.columns.droplevel(0)
paretopvt.columns.name = None
newdf = paretopvt.reset_index()
newdf

,Service,Olderthan14,Week,TTR Result\n(Excluding Pending Time),On-going,Pending,Resolved/ Closed
0,"3rd Party/ Client Controlled Software, Network...",Y,1,Beyond TAT,50.0,11.0,NaN
1,"3rd Party/ Client Controlled Software, Network...",Y,1,Within TAT,10.0,62.0,NaN
2,"3rd Party/ Client Controlled Software, Network...",Y,2,Beyond TAT,45.0,9.0,NaN
3,"3rd Party/ Client Controlled Software, Network...",Y,2,Within TAT,10.0,36.0,NaN
4,"3rd Party/ Client Controlled Software, Network...",Y,3,Beyond TAT,44.0,12.0,NaN
...,...,...,...,...,...,...,...
641,WFH PC setup,Y,23,Within TAT,8.0,10.0,2.0
642,WFH PC setup,Y,24,Beyond TAT,5.0,1.0,8.0
643,WFH PC setup,Y,24,Within TAT,3.0,4.0,8.0
644,WFH PC setup,Y,25,Beyond TAT,6.0,NaN,5.0


## Turn NaN values into 0 and Remove decimal using lambda

In [10]:
newdf['On-going'] = newdf['On-going'].fillna(0)
newdf['Pending'] = newdf['Pending'].fillna(0)
newdf['Resolved/ Closed'] = newdf['Resolved/ Closed'].fillna(0)

newdf['On-going'] = newdf['On-going'].astype(str).apply(lambda x: x.replace('.0',''))
newdf['Pending'] = newdf['Pending'].astype(str).apply(lambda x: x.replace('.0',''))
newdf['Resolved/ Closed'] = newdf['Resolved/ Closed'].astype(str).apply(lambda x: x.replace('.0',''))
newdf

,Service,Olderthan14,Week,TTR Result\n(Excluding Pending Time),On-going,Pending,Resolved/ Closed
0,"3rd Party/ Client Controlled Software, Network...",Y,1,Beyond TAT,50,11,0
1,"3rd Party/ Client Controlled Software, Network...",Y,1,Within TAT,10,62,0
2,"3rd Party/ Client Controlled Software, Network...",Y,2,Beyond TAT,45,9,0
3,"3rd Party/ Client Controlled Software, Network...",Y,2,Within TAT,10,36,0
4,"3rd Party/ Client Controlled Software, Network...",Y,3,Beyond TAT,44,12,0
...,...,...,...,...,...,...,...
641,WFH PC setup,Y,23,Within TAT,8,10,2
642,WFH PC setup,Y,24,Beyond TAT,5,1,8
643,WFH PC setup,Y,24,Within TAT,3,4,8
644,WFH PC setup,Y,25,Beyond TAT,6,0,5


Filter the current week

newdf = newdf.query('Week == 26') <-- make sure this is updated>

In [11]:
newdf = newdf.query('Week == 26')
newdf = newdf.rename( columns = {'TTR Result\n(Excluding Pending Time)': 'TTR'}, inplace = False)
nnewdf = newdf.query('TTR == "Beyond TAT"')
nnewdf

,Service,Olderthan14,Week,TTR,On-going,Pending,Resolved/ Closed
42,"3rd Party/ Client Controlled Software, Network...",Y,25,Beyond TAT,4,5,10
86,Access,Y,25,Beyond TAT,11,16,26
130,Email,Y,25,Beyond TAT,11,6,9
174,Hardware,Y,25,Beyond TAT,36,5,22
217,"IT software assessment, evaluation and testing",Y,25,Beyond TAT,2,0,1
260,IT software transferring,Y,25,Beyond TAT,0,0,2
351,Network or internet,Y,25,Beyond TAT,1,0,0
395,Onboarding Employee's IT-related Access and Ha...,Y,25,Beyond TAT,2,1,1
453,Resignee's Clearance Processing,Y,25,Beyond TAT,2,11,6
498,Server,Y,25,Beyond TAT,1,2,1


Add percentage column for beyond tat

In [12]:
sumongoingtat = nnewdf['On-going'].astype(int).sum()
ongoingpercent = nnewdf['On-going'].astype(int)/sumongoingtat
nnewdf["Ongoing TAT Percent"] = ongoingpercent

C:\Users\jewel.espiritu\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
nnewdf = nnewdf.rename( columns = {'On-going': 'On_going'}, inplace = False)
nnewdf.On_going = nnewdf.On_going.astype(float)
beyond = "On_going"
nnewdf = nnewdf.sort_values(beyond, ascending=False)

In [14]:
nnewdf['On_going'] = nnewdf['On_going'].astype(str).apply(lambda x: x.replace('.0',''))
nnewdf['cumulative_sum'] = nnewdf[beyond].astype(int).cumsum()
nnewdf['cumulative_perc'] = 100*nnewdf['cumulative_sum']/nnewdf[beyond].astype(int).sum()
nnewdf.to_excel(r'C:\Users\jewel.espiritu\OneDrive - AMDATEX LAS PINAS SERVICES INC\SLA\repo\pareto.xlsx', index = False, header = True)
nnewdf

,Service,Olderthan14,Week,TTR,On_going,Pending,Resolved/ Closed,Ongoing TAT Percent,cumulative_sum,cumulative_perc
174,Hardware,Y,25,Beyond TAT,36,5,22,0.409091,36,40.909091
563,Software,Y,25,Beyond TAT,12,3,17,0.136364,48,54.545455
86,Access,Y,25,Beyond TAT,11,16,26,0.125000,59,67.045455
130,Email,Y,25,Beyond TAT,11,6,9,0.125000,70,79.545455
644,WFH PC setup,Y,25,Beyond TAT,6,0,5,0.068182,76,86.363636
42,"3rd Party/ Client Controlled Software, Network...",Y,25,Beyond TAT,4,5,10,0.045455,80,90.909091
217,"IT software assessment, evaluation and testing",Y,25,Beyond TAT,2,0,1,0.022727,82,93.181818
395,Onboarding Employee's IT-related Access and Ha...,Y,25,Beyond TAT,2,1,1,0.022727,84,95.454545
453,Resignee's Clearance Processing,Y,25,Beyond TAT,2,11,6,0.022727,86,97.727273
351,Network or internet,Y,25,Beyond TAT,1,0,0,0.011364,87,98.863636


## Save a copy to local then send a copy to onedrive repo

In [ ]:
FilePath = r'C:\repo\SLA_Pareto.xlsx'
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(FilePath, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
nnewdf.to_excel(writer, sheet_name='Pareto', index = False)
writer.save()

